<a href="https://colab.research.google.com/github/STKKKKK/UTS_41004_ID12431868/blob/master/acp_a3_Group_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 41004_A3: Multi-Objective Optimization

Github Link:<br> https://github.com/STKKKKK/UTS_41004_ID12431868/blob/master/acp_a3_Group_30.ipynb

**Group #30 members:**<br>
Yi Lin   12444129<br>
Shiwei Xu   12418986<br>
Yuxiang Xu   12715948<br>
Hong Kung   12431868<br>

## Executive Summary & Problem Description

MOO refers to Multi-objective optimization. It is a field of multi criteria decision-making. It is a <br>
mathematical problem involving the simultaneous optimization of multiple objective functions. <br>

Our data analytics company will help our customer deal with MOO(multi-objective optimization) <br>problems with their dataset. As it is shown later, The dataset is about social network analysis,<br> include 1000 topic nodes (rows) and 5 attributes (columns). **In MOO, each attribute (column)<br>will be considered as a objective.**<br>

In the previous task, we successfully use 3 traditional approaches to re-rank these nodes by<br> multiple objectives, which both scalarization(equal weights) and machine learning appraoches <br>can only provide the result without practical significance in MOO aspect. Non-dominated sorting <br>performs better and provide the output which contains 37 pareto-fronts of 1000 nodes. Hence <br>we will develope the model based on non-dominated sorting.<br>

Our final goal is to build a NSGA(non-dominated sorting genetic algorithm) like model to process <br>data. The model detail will be dicussed in further sections. **Generally speaking, the input of <br> this project is the raw dataset, the output will be the ranking list of 1000 nodes.**<br>



## Data Exploration

We use the same dataset as previous, the dataset is an excel file and has been uploaded to github.  

In [0]:
import numpy as np
import pandas as pd
import math
import random

In [0]:
url = 'https://raw.githubusercontent.com/STKKKKK/UTS_41004_ID12431868/master/Topic4_IS_multipleRanking.xlsx'
df = pd.read_excel(url)

df.head()

,Node,Degree Centrality,Closeness Centrality,Between Centrality,Connectivity,Externality
0,Bibliometrics,3905.924925,0.806295,0.021293,9.620818,62045
1,Information retrieval,2157.933934,0.753394,0.018668,6.956349,26450
2,collaboration,2465.552553,0.793487,0.021247,6.073034,28579
3,United States,2077.891892,0.770833,0.018114,5.764045,23063
4,case study,1771.111111,0.794118,0.021508,2.436242,21383


Our NSGA model need easier access to specific columns or nodes in the dataset, and only <br>support numeric value analytics. Therefore, after we use pandas library to read the data, we need <br>to tranfer the data type from pandas.core.frame.DataFrame to a numpy array, then reserve it<br> and drop the Node-name column.

In [0]:
df = np.array(df)      
df = df.T[1:] 
# show the shape         
df.shape

(5, 1000)

## Reviewing & Planning

As previous task mentioned, NS(non-dominated sort) is not enough to deal with the MOO project since <br>we want a absolute ranking list instead of a list of pareto-fronts. We need to combine NS with a <br>robust algorithm model in MOO field. According to our research, GA(Generic algorithm) is ​used to <br>generate high-quality solutions to ‘​optimization​ and ​search problems​’, while NSGA (which is the <br>combination of NS and GA) is widely used in MOO,hence this will be the best choice of our project. <br>In other words, our goal is to build a model which is similar to NSGA.<br><br>


GA is inspired by the process of ​natural selection​.  Since our dataset doesn't have any off-spring issue.<br> We can't use the operators such as mutation, crossover and selection. In other words, we can't develop <br>an actual NSGA model or its advanced version such as NSGA-II. <br> 
Howover, we still need to find an operator to do re-rank all the nodes inside a pareto-front. **In that case,<br> we can add a crowding-distance operator (introduced in NSGA-II) to re-rank a single front.**<br><br>

**All in all,**<br>
**In previous we can only:** 
0. get the fronts, based on multiple columns of value.<br>

**In order to build our model, we still need:** <br>
1. sort a single front, based on
single column of value<br> 
2. sort a single front, based on a list of crowding distance.<br>
3. A crowding-distance calculator

## Modelling

**The description is shown in several """ docs.**

In [0]:
class NSGA:
    """
    This model is neither NSGA nor NSGA-II, but include some key functionalties 
    of them.
    """

    def __init__(self, data):
        self.data = data
        self.fronts = self.fast_non_dominated_sort()


    def run(self, pop_size): 
        """
        This is the main program to run the NSGA.
        This will return a ranking list of the nodes index.
        """
        ranks = []
        for front in self.fronts:
            cd = self.crowding_distance(front)
            new_front = self.sort_by_values(front, cd, val_type='dict')
            new_front.reverse()
            ranks.extend(new_front)
            if (len(ranks) > pop_size):
                break
        return ranks


    def dominate(self, a, b):   
        """
        This function will return an int.
        return 1:  true
        return 0:  false
        return -1: neither 
        a, b     : index of 2 points
        """    
        cols = self.data
        n = len(cols)
        flags = []
        for col in cols:
            if col[a] > col[b]:
                flags.append('greater')
            elif col[a] < col[b]:
                flags.append('smaller')
            elif col[a] == col[b]:
                flags.append('equal')
        
        if ('greater' not in flags and 'smaller' not in flags): # a,b are same point
            return -1
        elif ('smaller' not in flags):
            return 1
        elif ('greater' not in flags):
            return 0
        else:
            return -1


    def fast_non_dominated_sort(self):
        """
        This fuction will return the Pareto front after sorting.
        Front: Pareto front
        Dom:   dominate list of each point.
        n:     'inferior level' of each point.
            -- for example: n[A]==0 :point A is not dominated by any point.
                            n[A]==2 :point A is dominated by all points with n[B]==1,
                                    and dominates all points with n[C]==3.
        rank:  rank of the point.
        """
        cols = self.data
        Front = [[]]
        Dom = [[] for i in range(0,len(cols[0]))]
        n = [0 for i in range(0,len(cols[0]))]

        for f in range(0,len(cols[0])):
            for g in range(0, len(cols[0])):
                if (self.dominate(f, g) == 1) and not(g in Dom[f]):
                    Dom[f].append(g)
                elif (self.dominate(f, g) == 0):
                    n[f] = n[f] +1        
            if (n[f] == 0) and not(f in Front[0]):
                Front[0].append(f)

        rank = 0
        while(Front[rank] != []):
            Q=[]
            for f in Front[rank]:
                for g in Dom[f]:
                    n[g] = n[g] -1
                    if (n[g] == 0) and not(g in Q):
                        Q.append(g)
            rank = rank +1
            Front.append(Q)

        Front.pop()
        return Front 


    def sort_by_values(self, front, val, val_type='npArray'):
        """
        This function sort a single front from val.
        val: a list of values, which may according to: 
                1. a single column from dataframe (<numpy.ndarray>)
                2. or crowding distances (<dict>)
             Hence this function has 2 usages!
        """
        Sorted = []
        while (len(Sorted) != len(front)):
            if val_type == 'npArray':       
                val = list(val)   # numpy can't use .index()
                idx = val.index(min(val))
            elif val_type == 'dict':
                for key, dis in val.items():
                    if dis == min(val.values()):
                        idx = key
            else: return front   # substitude for using Exception 

            if idx in front:
                Sorted.append(idx)
            val[idx] = math.inf
        return Sorted


    def crowding_distance(self, front):
        """
        This function will return a dictionary of {nodes in front: their dis}. 
        dis: crowding_distance    
        For a single objective (column), the dis formula of a node:
            dis[node] = (o(larger neighbor) - o(smaller neighbor))/(max(o) – min(o))
            dis[node] = 0  if o(node) is the Min in its front, 
            dis[node] = 1  if o(node) is the Man in its front.
        """
        dis = {node: 0 for node in front}
        for col in self.data:
            Sorted = self.sort_by_values(front, col)
            for i, node in enumerate(Sorted[1:-1], start=1):
                dis[node] = dis[node] + (col[Sorted[i+1]]- col[Sorted[i-1]])/ (max(col)- min(col))
            # dis[Sorted[1]] = dis[Sorted[1]] + 0    # omitted
            dis[Sorted[-1]] = dis[Sorted[-1]] + 1
        return dis
        

## Data Processing

To process the dataset, we create the model.

In [0]:
model = NSGA(df)

Initially, we can test some separate methods of NSGA.

In [0]:
# Get the fronts, can also call 'fast_non_dominated_sort()'
fronts = model.fronts
print(*fronts[:7], sep='\n')
print('\n', 'There are', len(model.fronts), 'Pareto-fronts in 1000 nodes!')

[0, 4]
[1, 2]
[3, 7]
[5, 11, 6, 10, 12, 13, 16, 18, 43]
[29, 8, 14, 19, 24, 15, 21, 28]
[22, 32, 34, 35, 9, 33, 17, 20, 25, 26, 41]
[45, 39, 40, 44, 46, 23, 31, 37]

 There are 37 Pareto-fronts in 1000 nodes!


In [0]:
# show the crowding distance of each node in a front
model.crowding_distance(model.fronts[3])

{5: 2.0937400943931346,
 6: 0.44718699073326906,
 10: 0.20742885934900418,
 11: 0.3338109028915377,
 12: 0.17838242685781736,
 13: 2.147702358633937,
 16: 0.4455666263080032,
 18: 0.27357596915209775,
 43: 1.0302755697848434}

Then, we can run the model. <br>
if the pop_size is set to 20, we will get the node index which ranks from 1~20 out of 1000.

In [0]:
print(model.run(20))

[0, 4, 2, 1, 3, 7, 13, 5, 43, 6, 16, 11, 18, 10, 12, 8, 29, 19, 28, 15, 21, 24, 14]


## Future Prospects

Finally we see that the NSGA model run successfully. In fact, this model will support any size <br>of dataset without null values or non-numeric values, which the size is refers to columns(objectives) <br>and rows(nodes). <br>
This is why we recommend our 'NSGA' like model to customers. It is also important that we built <br>this NSGA class which provide the good extendability. Since there are still many operators need <br>to be added in order to build a completed NSGA-II though we don't need it for now. <br>At last, We expect that the developers like us will keep progress in future.